### This notebook covers some of the most fundamental concepts of tensors using Tensorflow 
#### We are going to cover:
* Introduction to Tensors
* Getting information from tensors
* Manipulation tensors
* Tensors and Numpy
* Using @tf.function (a way to speed up regular Python Functions)
* Using GPUs with TensorFlow (or TPUs)

### Introduction to Tensors

In [1]:
# Imprt Tensorflow
import tensorflow as tf  
import numpy as np
print(tf.__version__) 

2.16.1


In [2]:
# Create tensors with tf.constant() 
scalar = tf.constant(7) 
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
# Check the number of dimensions of a tensor (ndim stands for number of dimentions)
scalar.ndim

0

In [4]:
# Create a vector
vector = tf.constant([10,10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10])>

In [5]:
# Check the dimention of our vector
vector.ndim

1

In [6]:
# Create a matrix 
matrix = tf.constant([ 
    [10, 7],  
    [7,10] 
])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]])>

In [7]:
matrix.ndim

2

In [8]:
# Create another matrix
another_matrix = tf.constant([ 
    [10., 7.],                  
    [3., 2.],                         
    [8., 9.] 
], dtype=tf.float16) # Specify the data type with dtype paramater
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [9]:
another_matrix.ndim

2

In [10]:
# Let's create a tensor
tensor = tf.constant(
[ 
    [ 
        [1,2,3], 
        [4,5,6] 
    ], 
    [ 
        [7,8,9], 
        [10,11,12] 
    ], 
    [ 
        [13,14,15], 
        [16,17,18] 
    ] 
]
)
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]])>

In [11]:
tensor.ndim

3

### What we have created so far: 
* Scalar: a  single number 
* Vector: a number with direction (e.g. wind speed and direction)
* Matrix: a 2-dimensional array of numbers 
* Tensor: an n-dimensional array of numbers


### Creating tensors with tf.Variable

In [12]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10,7])  
unchangable_tensor = tf.constant([10, 7]) 
changeable_tensor, unchangable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7])>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7])>)

In [15]:
# Let's try to chage one of the elemts in our changeable tensor 
changeable_tensor[0] = 7 
changeable_tensor

TypeError: 'ResourceVariable' object does not support item assignment

In [16]:
# How about we try .assign() 
changeable_tensor[0].assign(7) 
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7])>

In [17]:
unchangable_tensor[0].assign(7)
unchangable_tensor

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

### Creating Random Tensors
Random tensors are  tensors of some arbitrary size that contain random numbers

In [18]:
# Create a random tensor (but use different distribution)

# Normal Distribution
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility 
random_1 = random_1.normal(shape=(3,2))

# Uniform Distribution
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.uniform(shape=(3,2))

# Are they equal (no since they are different distribution)
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[0.7493447 , 0.73561966],
        [0.45230794, 0.49039817],
        [0.1889317 , 0.52027524]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[False, False],
        [False, False],
        [False, False]])>)

### Shuffle the order of elements in a Tensor 

In [19]:
# Shuffle a tensor (Valuable for when you want to shuffle your data so the inherent order doesn't effect learning) 
not_shuffled = tf.constant([
    [10, 7], 
    [3, 4], 
    [2, 5]
]) 
# Shuffle our non  shuffled tensor
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [ 3,  4],
       [10,  7]])>

### Other ways to make tensors

In [20]:
# Create a tensor of all ones
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [21]:
# Create a tensor of all zeros
tf.zeros(shape=(3,4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn NumPy arrays into tensors  
The main difference between Numpy array and TensorFlow tensors is that tensors can be run on a GPU (mutch faster for numerical computing)"


In [22]:
import numpy as np 

In [23]:
# Turining a NumPy array into tensors
numpy_A = np.arange(1, 25, dtype=np.int32) # Create a Numpy array between 1 and 25
numpy_A
# x = tf.constant(some_matrix) # Capital for matrix or tensor
# y = tf.constant(vector) # Non capital for vectors

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [24]:
# We can just pass in the NumPy array into tf.constant
A = tf.constant(numpy_A) 
B = tf.constant(numpy_A, shape=(2,3,4)) 
A, B 
# Notice how the shape change needs to be valid

(<tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24])>,
 <tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]])>)

### Getting information from tensors 
When dealing with tensors we will probably need to be aware of the following attributes
* Shape
* Rank
* Axis or Dimension
* Size

In [25]:
# Create a rank 4 tensor (4 dimentions) 
rank_4_tensor = tf.zeros(shape=[2,3,4,5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [26]:
rank_4_tensor[0] # 0th index

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [27]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [28]:
# Get various attributes of our tensors 
print("Datatype of every element:", rank_4_tensor.dtype) 
print("Number of dimensions (rank)", rank_4_tensor.ndim) 
print("Shape of our tensor:", rank_4_tensor.shape) 
print("Elements along 0th axis:", rank_4_tensor.shape[0])  
print("Elements along the last axis",rank_4_tensor.shape[-1]) 
print("The total number of elements in our tensor", tf.size(rank_4_tensor)) 
print("The total number of elements in our tensor (With Numpy as single element)", tf.size(rank_4_tensor).numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank) 4
Shape of our tensor: (2, 3, 4, 5)
Elements along 0th axis: 2
Elements along the last axis 5
The total number of elements in our tensor tf.Tensor(120, shape=(), dtype=int32)
The total number of elements in our tensor (With Numpy as single element) 120


### Indexing tensors 
Tensors can be indexed just like Python lists

In [29]:
rank_4_tensor[:2, :2, :2, :2] # First 2 element of  each dimention

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [30]:
# Get the first element from each dimension from each index except for the second last one
rank_4_tensor[:1, :1 ,: , :1]

<tf.Tensor: shape=(1, 1, 4, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.],
         [0.]]]], dtype=float32)>

In [31]:
# Create a rank 2 tensor (2 dimentions) 
rank_2_tensor = tf.constant([
    [10, 7], 
    [3, 4],
]) 
rank_2_tensor.shape, rank_2_tensor.ndim

(TensorShape([2, 2]), 2)

In [32]:
# Get the last item of each row of our rank 2 tensor
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4])>

In [33]:
# Add in extra dimentions to our rank 2 tensor 
rank_2_tensor = rank_2_tensor[..., tf.newaxis] # ... means on each previous axis  (rank_2_tensor[:, :, tf.newaxis])
rank_2_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]])>

In [34]:
# Alternative to tf.newaxis 
tf.expand_dims(rank_2_tensor, axis=-1) # "-1" means expand the final axis

<tf.Tensor: shape=(2, 2, 1, 1), dtype=int32, numpy=
array([[[[10]],

        [[ 7]]],


       [[[ 3]],

        [[ 4]]]])>

### Manipulating Tensors 
*Basic Operations:* 
( +, -, /, *)

In [35]:
# We can add values to a tensor using the addition operator
tensor = tf.constant([
    [10, 7], 
    [3, 4]
]) 

tensor + 10 # Add 10 to everything (Original tensor remains unchanged)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]])>

In [36]:
tensor = tensor + 10 # This will change it 
tensor 
# Multiplication, and Subtraction work in the same way

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]])>

In [37]:
# We can use the TensorFlow built-in function too 
tf.math.multiply(tensor, 10)  # Still unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]])>

### Matrix Multiplication 
One of the most common tensor operations

Recall that there are 2 rules for valid matrix multiplication 
1. The inner dimensions must match
2. The resulting matrix has the shape of the outer dimensions

In [38]:
# Matrix Multiplication in tensorflow 
tf.matmul(tensor, tensor) # tensor times tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[621, 578],
       [442, 417]])>

In [39]:
# Create a tensor of 3 x 2 
X = tf.constant([
    [1, 2], 
    [3, 4], 
    [5, 6]
]) 

# Create abother 3 x 2 tesnor 
Y = tf.constant([
    [7, 8], 
    [9, 10], 
    [11, 12]
])  

X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]])>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]])>)

In [40]:
# Try to matrix multiply tensors of same shape
tf.matmul(X,Y)

InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2] [Op:MatMul] name: 

In [ ]:
# Lets reshape Y to fix this 
tf.reshape(Y, shape=(2,3))

In [ ]:
# Lets multiply now (@ does matrix multiplication)
X @ tf.reshape(Y, shape=(2,3))

In [ ]:
tf.matmul(X, tf.reshape(Y, shape=(2,3))) # Same thing

In [ ]:
# What we reshape X?? 
tf.matmul(tf.reshape(X, shape=(2,3)), Y)

In [ ]:
# Can do the same with transpose (Flips the axis) 
# Transpose of a matrix flips rows and columns and vice versa
X, tf.transpose(X), tf.reshape(X, shape=(2,3))

In [ ]:
# Try matrix mulitplication with teranspose rtather than reshape 
tf.matmul(tf.transpose(X), Y)

### The dot product

Matrix multiplication is the same thing as the dot product: 
* `tf.matmul()`
* `tf.tensordot()`

In [41]:
# Preform the dot product on X and Y  
tf.tensordot(tf.transpose(X), Y, axes = 1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]])>

In [42]:
# Preform matrix multiplication between X and Y (transposed) 
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]])>

In [43]:
# Preform matrix multiplication between X and Y (reshaped)  
tf.matmul(X, tf.reshape(Y, shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]])>

In [44]:
# Check the values of Y, reshape Y and transpose Y 
print("Normal Y:") 
print(Y, "\n")

print("Y resphaed to (2,3)):") 
print(tf.reshape(Y, (2,3)), "\n")

print("Y transposed:") 
print(tf.transpose(Y))

Normal Y:
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y resphaed to (2,3)):
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32) 

Y transposed:
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32)


##### Generally when performing matrix multiplication on two tensors and one of the axes doesn't line up, we will transpose (rather than reshape) one of the tensors to satisfy the matrix multiplication rules

### Changing the Datatype of a Tensor

In [45]:
# Create a new tensore with default datatype (float32) 
B = tf.constant([1.7, 7.4])  
B, B.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.7, 7.4], dtype=float32)>,
 tf.float32)

In [46]:
C = tf.constant([7, 10]) 
C, C.dtype

(<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10])>, tf.int32)

In [47]:
# What if we want to change from float32 to float16
D = tf.cast(B, dtype=tf.float16)
D, D.dtype

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 tf.float16)

In [48]:
# Change from int32 to float32 
E = tf.cast(C, dtype=tf.float32) 
E, E.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>,
 tf.float32)

### Aggregating Tensors 
Aggregating tensors means condensing them from multiple vlaues down to a smaller ammount of values

In [49]:
D = tf.constant([-7, -10])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10])>

In [50]:
# Get the absolute values  
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10])>

#### Lets go through the follwing forms of aggregation: 
* Get the minimum
* Get the maximum
* Get the mean of a tensor
* Get the sum of a tensor
* Get the variance of our tensor
* Get the standard deviation of our tensor

In [51]:
# Create a random tensor with values between 0 and 100 of size 50 
E = tf.constant(np.random.randint(0, 100, size=50))
E

<tf.Tensor: shape=(50,), dtype=int32, numpy=
array([28, 30, 31, 56, 41, 50, 92, 72, 58, 17, 90,  3, 67, 93, 99, 64,  1,
       12, 31, 75, 68, 61, 93, 45,  3, 42, 47, 94, 59, 86, 84, 25, 95, 76,
       90, 82, 18, 78, 98, 92, 61, 30, 74, 55,  6, 99, 72, 30,  7, 54])>

In [52]:
tf.size(E), E.shape, E.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [53]:
# Find the minimum 
tf.reduce_min(E)

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [54]:
# Find the maximum 
tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=int32, numpy=99>

In [55]:
# Find the mean 
tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int32, numpy=56>

In [56]:
# Find the sum 
tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int32, numpy=2834>

In [58]:
import tensorflow_probability as tfp
tfp.stats.variance(E)

<tf.Tensor: shape=(), dtype=int32, numpy=900>

In [66]:
# Find the standard deviation
tf.math.reduce_std(tf.cast(E, dtype=tf.float32)) # Needs to be a float or else wont work!

<tf.Tensor: shape=(), dtype=float32, numpy=30.003626>

In [68]:
# Find the varaince in a simple way 
tf.math.reduce_variance(tf.cast(E, dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=900.2176>

### Finding the positional maximum and minimum

In [70]:
# Create a new tensor for finding positional minimum and maximum 
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [73]:
# Find the positional maximum (index of the max) 
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [74]:
# Index on our largies value position 
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [75]:
# Find the max value of F
tf.reduce_max(F) # Wow its the same!

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [77]:
# Check for equality 
F[tf.argmax(F)] == tf.reduce_max(F) # Check

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [78]:
# Find the positional minimum 
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [80]:
F[tf.argmin(F)] 
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

### Squeezing a tensor (reoving all single dimentions)

In [83]:
# Create a tensor to get started  
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50)) 
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [84]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [87]:
G_squeezed = tf.squeeze(G) # Removes all dimensions of size 1 
G_squeezed # Now its size 50

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

### One hot encoding tensors  
We can one hot encode catagorical data sutch as words before passing into neural network since computer cant understand words

In [88]:
# Create a list of indices 
some_list = [0, 1, 2, 3] # Could be red green blue purple 

# One hot encode our list of indices 

tf.one_hot(some_list, depth=4) # depth = 4 since we have 4 elements

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [92]:
# Specify custom values for one hot encoding 
tf.one_hot(some_list,  depth=4, on_value="This", off_value="That")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'This', b'That', b'That', b'That'],
       [b'That', b'This', b'That', b'That'],
       [b'That', b'That', b'This', b'That'],
       [b'That', b'That', b'That', b'This']], dtype=object)>

### Squaring, log and square root

In [94]:
# Create a new tensor 
H = tf.range(1, 10) 
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9])>

In [95]:
# Square 
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81])>

In [100]:
# Square root 
tf.sqrt(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.236068 , 2.4494898,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [103]:
# Find the log 
tf.math.log(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### Tensorflow and Numpy 
Tensorflow interacts beautifully with NumPy array

In [105]:
# Create a tensor directly from a NumPy array 
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [106]:
# Convert our tensor back to a NumPy array 
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [107]:
# Convert tensor J back into a NumPy array  
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [110]:
# The default types of each are slightly different 
numpy_J = tf.constant(np.array([3. ,7., 10.])) 
tensor_J = tf.constant([3. ,7., 10.])  

# Check the datatypes of each (Notice how the default types are a bit different)
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

In [111]:
# Note that a tensorflow tensor can be run on a GPU or TPU

### Finding acess to GPU's 

In [115]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]